In [ ]:
! pip uninstall -y tensorflow_datasets
! pip install tensorflow_datasets==4.4.0

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.preprocessing import LabelEncoder
import dill
import tensorflow as tf
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import backend as K 
import seaborn as sns
import random
import gc
from tqdm.notebook import tqdm
import tensorflow_addons as tfa
import sys
sys.path.append("../input/sorghum100cultivarjpgimages512x512") 
# from fgvc_dataset import FGVCDataset

In [ ]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code
tpu = None
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

    
#strategy,tpu = tf.distribute.MirroredStrategy(devices=["TPU:0", "TPU:1","TPU:2"]),True

print("Number of accelerators: ", strategy.num_replicas_in_sync)


AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:

from sklearn.preprocessing import  LabelEncoder
from tqdm.auto import tqdm
import tensorflow as tf
import dill
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import glob

In [ ]:
# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# SciKit Learn
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score, make_scorer, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split

from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize 
from sklearn.feature_selection import SelectKBest, f_classif

# TensorFlow & Keras
import tensorflow as tf
# from tensorflow import keras
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.applications import EfficientNetB0 , InceptionV3 ,NASNetLarge,NASNetMobile, ResNet50, ResNet50V2, Xception

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


# Misc
import random
import warnings
from sys import modules
from time import time
import os
from platform import python_version
import urllib #<! Download internet files
import shutil #<! Handle zip files

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# IPython
from IPython.display import Image, display
from IPython.core.display import HTML
from IPython import get_ipython

# Confuguration
%matplotlib inline
mpl.style.use('ggplot' )
plt.style.use('fivethirtyeight')
sns.set(context = "notebook", palette = "dark", style = 'whitegrid', color_codes = True)

In [ ]:
print(dir(tf.keras.applications))

In [ ]:
train_df = pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")

In [ ]:
test_df = pd.DataFrame({
    'filename':[os.path.basename(name) for name in glob.glob("../input/sorghum-id-fgvc-9/test/*.png")]
})
test_df['cultivar'] = ''
test_df.to_csv("submission.csv",index=False)

In [ ]:
test_df.head(5)


In [ ]:
train_df['jpeg_image'] = [path[:-3]+"jpg" for path in train_df.image]
test_df['jpeg_image'] = [path[:-3]+"jpg" for path in test_df.filename]


In [ ]:
def image_exists(path):
    return os.path.exists(path)
train_df = train_df[[image_exists(f"../input/sorghum-id-fgvc-9/train_images/{img}") for img in train_df.image]]

In [ ]:
train_df.shape,test_df.shape

In [ ]:
le = LabelEncoder()
le = le.fit(train_df.cultivar)
dill.dump(le,open("le.dill",'wb'))
train_df["target"] = le.transform(train_df.cultivar)

In [ ]:
print (test_df.shape)
train_df.shape

## Creating Dataset


In [ ]:
# os.remove("../input/mytffilessorghum/fgvc_dataset.py")
os.remove("./fgvc_dataset.py")


In [ ]:
shutil.rmtree("./fgvc_dataset_train_val_test")

In [ ]:
from subprocess import check_output

print(check_output(["ls", "./fgvc_dataset_train_val_test/fgvc_dataset/0.1.0"]).decode("utf8"))


In [ ]:
%%writefile fgvc_dataset.py -a

import tensorflow_datasets as tfds
import tensorflow as tf

class FGVCDataset(tfds.core.GeneratorBasedBuilder):
    VERSION = tfds.core.Version('0.1.0')
    
    def _split_generators(self, dl_manager):
        arr = [
            tfds.core.SplitGenerator(name=f'train',gen_kwargs={"split":"train"}),
            tfds.core.SplitGenerator(name=f'test',gen_kwargs={"split":"test"}),
            tfds.core.SplitGenerator(name=f'val',gen_kwargs={"split":"val"})
        ]
        return arr
    
    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=(""),
            #disable_shuffling=True,
            features=tfds.features.FeaturesDict({
                "img": tfds.features.Image(encoding_format='jpeg'),#dtype=tf.uint8,shape=(self.WIDTH,self.HEIGHT,3),
                "name": tfds.features.Tensor(dtype=tf.string,shape=()),
                "cultivar": tfds.features.Tensor(dtype=tf.string,shape=()),
                "target": tfds.features.Tensor(dtype=tf.int32,shape=()),
            }),
        )
    
    def _generate_examples(self,**args):
        print(args)
        split = args["split"]
        
        if split == 'train':
            for i in range(len(self.train_df)):
                row = self.train_df.iloc[i]
                img = f"../input/sorghum100cultivarjpgimages512x512/train_images/{row.jpeg_image}"
                yield i, {
                    'img':img,
                    'cultivar':row.cultivar,
                    'name':row.image,
                    'target':row.target,
                }
                
        if split == 'val':
            for i in range(len(self.val_df)):
                row = self.val_df.iloc[i]
                img = f"../input/sorghum100cultivarjpgimages512x512/train_images/{row.jpeg_image}"
                yield i, {
                    'img':img,
                    'cultivar':row.cultivar,
                    'name':row.image,
                    'target':row.target,
                }
        if split == 'test':
            for i in range(len(self.test_df)):
                row = self.test_df.iloc[i]
                img = f"../input/sorghum100cultivarjpgimages512x512/test/{row.jpeg_image}"
                yield i, {
                    'img':img,
                    'name':row.filename,
                    'cultivar':'',
                    'target':-1,
                }

In [ ]:
# %%writefile fgvc_dataset.py -a

# import tensorflow_datasets as tfds
# import tensorflow as tf

# class FGVCDataset(tfds.core.GeneratorBasedBuilder):
#     VERSION = tfds.core.Version('0.1.0')
    
#     def _split_generators(self, dl_manager):
#         arr = [
#             tfds.core.SplitGenerator(name=f'train',gen_kwargs={"split":"train"}),
#             tfds.core.SplitGenerator(name=f'test',gen_kwargs={"split":"test"})
#         ]
#         return arr
    
#     def _info(self):
#         return tfds.core.DatasetInfo(
#             builder=self,
#             description=(""),
#             #disable_shuffling=True,
#             features=tfds.features.FeaturesDict({
#                 "img": tfds.features.Image(encoding_format='jpeg'),#dtype=tf.uint8,shape=(self.WIDTH,self.HEIGHT,3),
#                 "name": tfds.features.Tensor(dtype=tf.string,shape=()),
#                 "cultivar": tfds.features.Tensor(dtype=tf.string,shape=()),
#                 "target": tfds.features.Tensor(dtype=tf.int32,shape=()),
#             }),
#         )
    
#     def _generate_examples(self,**args):
#         print(args)
#         split = args["split"]
        
#         if split == 'train':
#             for i in range(len(self.train_df)):
#                 row = self.train_df.iloc[i]
#                 img = f"../input/sorghum100cultivarjpgimages512x512/train_images/{row.jpeg_image}"
#                 yield i, {
#                     'img':img,
#                     'cultivar':row.cultivar,
#                     'name':row.image,
#                     'target':row.target,
#                 }
                
#         if split == 'test':
#             for i in range(len(self.test_df)):
#                 row = self.test_df.iloc[i]
#                 img = f"../input/sorghum100cultivarjpgimages512x512/test/{row.jpeg_image}"
#                 yield i, {
#                     'img':img,
#                     'name':row.filename,
#                     'cultivar':'',
#                     'target':-1,
#                 }

In [ ]:
from fgvc_dataset import FGVCDataset


In [ ]:
train_df, val_df = train_test_split(train_df,
    test_size=0.2, shuffle = True, random_state = 8)

In [ ]:
%%time
data_dir='./fgvc_dataset_train_val_test' 
FGVCDataset.val_df = val_df#.iloc[:100]
FGVCDataset.train_df = train_df#.sample(frac =.60)
FGVCDataset.test_df = test_df#.iloc[:100]

builder = FGVCDataset(data_dir=data_dir)
builder.download_and_prepare() 

In [ ]:
train_ds = builder.as_dataset()['train']
test_ds = builder.as_dataset()['test']
val_ds = builder.as_dataset()['val']

In [ ]:
# builder.as_dataset()


In [ ]:
# train_ds = builder.as_dataset()['train']
# test_ds = builder.as_dataset()['test']

In [ ]:
# for x in train_ds.take(1):
#     img = x['img']
#     cultivar = x['cultivar']
#     target = x['target']
#     print(img.shape,cultivar,target)


In [ ]:
# plt.imshow(img.numpy())


In [ ]:
# train_df = pd.read_csv("../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv")
# test_df = pd.read_csv("../input/sorghum-id-fgvc-9/sample_submission.csv")
GCS_PATH = KaggleDatasets().get_gcs_path('mytffilessorghum') 

In [ ]:
GCS_PATH

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.dropna(axis=0,inplace=True)

In [ ]:
USE_SAVED_MODEL = False


In [ ]:
%%time
data_dir= GCS_PATH+"/fgvc_dataset512" 
builder = FGVCDataset(data_dir=data_dir)
builder.download_and_prepare()
train_ds = builder.as_dataset()['train']
test_ds = builder.as_dataset()['test']

In [ ]:
train_ds

In [ ]:
def show_image_batch(images: list):
    """
    Displays a batch of image present in images
    """
    fig = plt.figure(figsize=(10,5))
    for idx in range(6):
        ax = plt.subplot(2, 3, idx+1)
        plt.imshow(images[idx])
        plt.axis("off")

def show_dataset(dataset):
    batch = next(iter(dataset))
    images, labels = batch
    
    plt.figure(figsize=(10, 10))
    for idx in range(9):
        ax = plt.subplot(3, 3, idx + 1)
        plt.imshow(images[idx].numpy().astype("uint8"))
        plt.title("Class: {}".format(labels[idx].numpy().decode()))
        plt.axis("off")

In [ ]:
show_dataset(train_ds.map(lambda data:(data['img'],data['cultivar'])).batch(9))

In [ ]:
le = dill.load(open("./le.dill","rb"))
train_df["target"] = le.transform(train_df.cultivar)

In [ ]:
SEED = 123
HEIGHT = 512
WIDTH = 512
N_CLASSES = len(train_df.cultivar.unique())
BATCH_SIZE = 32 if tpu else 4
IMG_DIR = "../input/mytffilessorghum" if not tpu else GCS_PATH

In [ ]:
IMG_DIR

# New trials

In [ ]:
def data_augmentation():
    return keras.Sequential([
        #keras.layers.experimental.preprocessing.RandomZoom(0.2,seed=SEED),
        #keras.layers.experimental.preprocessing.RandomCrop(HEIGHT//2, WIDTH//2,seed=SEED),
        keras.layers.experimental.preprocessing.RandomContrast(0.1,seed=SEED),
        keras.layers.experimental.preprocessing.RandomFlip(),
        keras.layers.experimental.preprocessing.RandomRotation(0.2,seed=SEED),
        #keras.layers.experimental.preprocessing.Resizing(HEIGHT, WIDTH),
    ])
daug = data_augmentation()

In [ ]:
def load_train_image(data):
    img = data['img']
    cultivar = data['cultivar']
    name = data['name']
    target = data['target']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)
    return img,target

def load_test_image(data):
    img = data['img']
    name = data['name']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img,name

def prepare_train_dataset(train_ds):
    steps = len(train_ds)//BATCH_SIZE
    train_ds = train_ds.repeat().shuffle(5000).map(load_train_image,num_parallel_calls=AUTO)
    train_ds = train_ds.batch(BATCH_SIZE).map(lambda x,y:(daug(x),y),num_parallel_calls=AUTO).prefetch(100)
    return steps,train_ds

def prepare_submission_dataset(ds):
    ds = ds.map(load_test_image,num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE*2).prefetch(100)
    return ds

In [ ]:
show_dataset(train_ds.map(lambda data:(data['img'],data['cultivar'])).batch(9).map(lambda x,y:(daug(x),y)))

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

scaleLayer = Rescaling(scale = 1 / 127.5, offset = -1)

In [ ]:

# Models are bottom (Input) up (Output)
# We could take the top layer and deifne number of classes for 2
# For practice we do it manually.
baseModel = keras.applications.Xception(
    weights = "imagenet",  #<! Load weights pre-trained on ImageNet. One could use a file path!
    input_shape = (None, None, 3),
    include_top = False, #<! Last layer (The classifier for 1000 classes)
)  #<! Do not include the ImageNet classifier at the top.

baseModel.trainable = False #<! Freeze the base model
baseModel.summary()

In [ ]:
inputSize = (512, 512)


In [ ]:
# out = backbone(image_input)
# # out = tf.keras.layers.GlobalAveragePooling2D()(out)
# # out = tf.keras.layers.Dense(N_CLASSES,activation="softmax")(out)

In [ ]:


# Create the model on top
inputLayer = keras.Input(shape = (*inputSize, 3))
x = daug(inputLayer) #<! Apply random data augmentation
x = scaleLayer(x) #<! Re Scaling of the data

# The base model contains `batchNorm` layers. We want to keep them in inference mode
# when we unfreeze the base model for fine tuning, so we make sure that the
# base_model is running in inference mode here.

x = baseModel(x, training = False) #<! Think of the Dropout / Batch Normalization
x = baseModel(inputLayer)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)  #<! Regularization by dropout
outputLayer = tf.keras.layers.Dense(N_CLASSES,activation="softmax")(x) #<! No need for activation (Sigmoid) since we use Binary Cross Entropy with `from_logit`
SorgModel = keras.Model(inputLayer, outputLayer)

SorgModel.summary()

In [ ]:
# with strategy.scope():
#     image_input = tf.keras.layers.Input(shape=(WIDTH,HEIGHT,3))
    
#     backbone = tf.keras.applications.Xception(include_top=False,weights="imagenet",input_shape=(WIDTH,HEIGHT,3))
#     out = backbone(image_input)
#     out = tf.keras.layers.GlobalAveragePooling2D()(out)
#     out = tf.keras.layers.Dense(N_CLASSES,activation="softmax")(out)
    
#     model = tf.keras.Model(image_input,out)
#     model.compile()
#     model.summary()

In [ ]:
# SorgModel.compile(
#     optimizer = Adam(), #<! In pratcice we should set a schedule
#     loss = BinaryCrossentropy(from_logits = True),
#     metrics = [BinaryAccuracy()],
# )

# numEpochs = 2

# catDogModel.fit(dsTrain, epochs = numEpochs, validation_data = dsValidation)

In [ ]:
with strategy.scope():
    if not USE_SAVED_MODEL:
        steps_per_epoch,ds =  prepare_train_dataset(train_ds)
        callback = tf.keras.callbacks.EarlyStopping(monitor='acc',mode='max', patience=20)
        ckp_callback = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath=f'model.h5',
                                                    save_weights_only=True,
                                                    monitor='acc',
                                                    mode='max',
                                                    options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'),
                                                    save_best_only=True)
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='acc',mode='max',factor=0.2,patience=3, min_lr=1e-5)
        callbacks=[callback,ckp_callback,reduce_lr]
        # Compile the model
        SorgModel.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                      loss=tf.keras.losses.sparse_categorical_crossentropy,
                      metrics=['acc',])

        history = SorgModel.fit(ds, 
                            steps_per_epoch=steps_per_epoch,
                            epochs=30,
                            verbose=1,
                            callbacks=callbacks)

# Predict

In [ ]:
if not USE_SAVED_MODEL:
    model.load_weights('model.h5')
else:
    model.load_weights('../input/sorghum100efficientnetbaselinemodel/model.h5')

In [ ]:
def predict(x):
    return model(x,training=False)
@tf.function
def dist_predict(dist_inputs):
    res = strategy.run(predict, args=(dist_inputs,))
    return res

In [ ]:
%%time
with strategy.scope():
    ds = prepare_submission_dataset(test_ds)
    dist_ds = strategy.experimental_distribute_dataset(ds)
    
    all_names = []
    all_targets = []
    for img,names in tqdm(dist_ds):
        preds = dist_predict(img)
        if tpu:
            preds = tf.concat(preds.values,axis=0)
            names = tf.concat(names.values,axis=0)
        preds = preds.numpy()
        names = names.numpy()
        preds = np.argmax(preds,axis=-1)
        all_targets.extend(list(preds))
        all_names.extend([s.decode('ascii') for s in names])

In [ ]:
sub_df = pd.DataFrame({
    "filename":all_names,
    "cultivar":le.inverse_transform(all_targets)
})
sub_df.head(5)

In [ ]:
sub_df.to_csv("submission.csv",index=False)


# original model

In [ ]:
def data_augmentation():
    return keras.Sequential([
        #keras.layers.experimental.preprocessing.RandomZoom(0.2,seed=SEED),
        #keras.layers.experimental.preprocessing.RandomCrop(HEIGHT//2, WIDTH//2,seed=SEED),
        keras.layers.experimental.preprocessing.RandomContrast(0.1,seed=SEED),
        keras.layers.experimental.preprocessing.RandomFlip(),
        keras.layers.experimental.preprocessing.RandomRotation(0.2,seed=SEED),
        #keras.layers.experimental.preprocessing.Resizing(HEIGHT, WIDTH),
    ])
daug = data_augmentation()

In [ ]:
def load_train_image(data):
    img = data['img']
    cultivar = data['cultivar']
    name = data['name']
    target = data['target']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)
    return img,target

def load_test_image(data):
    img = data['img']
    name = data['name']
    # Resize image
    img = tf.image.resize(img,(WIDTH, HEIGHT),)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img,name

def prepare_train_dataset(train_ds):
    steps = len(train_ds)//BATCH_SIZE
    train_ds = train_ds.repeat().shuffle(5000).map(load_train_image,num_parallel_calls=AUTO)
    train_ds = train_ds.batch(BATCH_SIZE).map(lambda x,y:(daug(x),y),num_parallel_calls=AUTO).prefetch(100)
    return steps,train_ds

def prepare_submission_dataset(ds):
    ds = ds.map(load_test_image,num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE*2).prefetch(100)
    return ds

In [ ]:
show_dataset(train_ds.map(lambda data:(data['img'],data['cultivar'])).batch(9).map(lambda x,y:(daug(x),y)))

In [ ]:
with strategy.scope():
    image_input = tf.keras.layers.Input(shape=(WIDTH,HEIGHT,3))
    
    backbone = tf.keras.applications.EfficientNetB4(include_top=False,weights="imagenet",input_shape=(WIDTH,HEIGHT,3))
    out = backbone(image_input)
    out = tf.keras.layers.GlobalAveragePooling2D()(out)
    out = tf.keras.layers.Dense(N_CLASSES,activation="softmax")(out)
    
    model = tf.keras.Model(image_input,out)
    model.compile()
    model.summary()

In [ ]:
with strategy.scope():
    if not USE_SAVED_MODEL:
        steps_per_epoch,ds =  prepare_train_dataset(train_ds)
        callback = tf.keras.callbacks.EarlyStopping(monitor='acc',mode='max', patience=20)
        ckp_callback = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath=f'model.h5',
                                                    save_weights_only=True,
                                                    monitor='acc',
                                                    mode='max',
                                                    options=tf.train.CheckpointOptions(experimental_io_device='/job:localhost'),
                                                    save_best_only=True)
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='acc',mode='max',factor=0.2,patience=3, min_lr=1e-5)
        callbacks=[callback,ckp_callback,reduce_lr]
        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                      loss=tf.keras.losses.sparse_categorical_crossentropy,
                      metrics=['acc',])

        history = model.fit(ds,
                            steps_per_epoch=steps_per_epoch,
                            epochs=30,
                            callbacks=callbacks)

In [ ]:
if not USE_SAVED_MODEL:
    model.load_weights('model.h5')
else:
    model.load_weights('../input/sorghum100efficientnetbaselinemodel/model.h5')

In [ ]:
def predict(x):
    return model(x,training=False)
@tf.function
def dist_predict(dist_inputs):
    res = strategy.run(predict, args=(dist_inputs,))
    return res

In [ ]:
%%time
with strategy.scope():
    ds = prepare_submission_dataset(test_ds)
    dist_ds = strategy.experimental_distribute_dataset(ds)
    
    all_names = []
    all_targets = []
    for img,names in tqdm(dist_ds):
        preds = dist_predict(img)
        if tpu:
            preds = tf.concat(preds.values,axis=0)
            names = tf.concat(names.values,axis=0)
        preds = preds.numpy()
        names = names.numpy()
        preds = np.argmax(preds,axis=-1)
        all_targets.extend(list(preds))
        all_names.extend([s.decode('ascii') for s in names])

In [ ]:
sub_df = pd.DataFrame({
    "filename":all_names,
    "cultivar":le.inverse_transform(all_targets)
})
sub_df.head(5)

In [ ]:
sub_df.to_csv("submission.csv",index=False)
